<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/main/HW12/HW12_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「環境」給予的 reward 大致是這樣計算：
- 小艇墜毀得到 -100 分
- 小艇在黃旗幟之間成功著地則得 100~140 分
- 噴射主引擎（向下噴火）每次 -0.3 分
- 小艇最終完全靜止則再得 100 分
- 小艇每隻腳碰觸地面 +10 分
- 噴射左或右引擎（順時針或逆時針旋轉）每次 -0.03 分

###Setting



###Install Package



In [1]:
!apt update
!apt install python3-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install swig
!pip install gym[box2d]
!pip install tqdm
!pip install box2d-kengz

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [805 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,124 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/

###Set Seed

In [2]:
import torch
import numpy as np
import random
seed = 543
def fix(env, seed):
  env.seed(seed)
  env.action_space.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

###Pretrain
######Using policy gradient to pretrain model to get a better Q.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
class PolicyGradientNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, state):
        hid = torch.tanh(self.fc1(state))
        hid = torch.tanh(self.fc2(hid))
        hid = torch.tanh(self.fc3(hid))
        hid = torch.tanh(self.fc4(hid))
        return self.fc5(hid)

In [4]:
import torch.optim as optim
from torch.distributions import Categorical
class PolicyGradientAgent():

    def __init__(self, network):
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=0.001)

    def learn(self, log_probs, rewards):

        loss = (-log_probs * rewards).sum()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def sample(self, state):
        action_prob = F.softmax(self.network(torch.FloatTensor(state)), dim=-1)
        action_dist = Categorical(action_prob)
        action = action_dist.sample()
        log_prob = action_dist.log_prob(action)
        return action.item(), log_prob

In [5]:
from tqdm.notebook import tqdm
def policy_gradient_training(env,agent):
    agent.network.train()
    EPISODE_PER_BATCH = 5
    NUM_BATCH = 800
    avg_total_rewards, avg_final_rewards = [], []

    prg_bar = tqdm(range(NUM_BATCH))
    for batch in prg_bar:

        log_probs, rewards = [], []
        total_rewards, final_rewards = [], []

        for episode in range(EPISODE_PER_BATCH):

            state = env.reset()
            total_reward, total_step = 0, 0
            seq_rewards = []
            while True:

                action, log_prob = agent.sample(state) # at(int) , log(at|st)
                next_state, reward, done, _ = env.step(action)

                log_probs.append(log_prob) # [log(a1|s1), log(a2|s2), ...., log(at|st)]
                state = next_state
                total_reward += reward
                total_step += 1

                seq_rewards.append(0)
                len_of_seq_reward = len(seq_rewards)
                seq_rewards = [seq_rewards[index] +(0.99)**(len_of_seq_reward-(index+1))\
                        *reward for index in range(len_of_seq_reward)]
                if done:
                    rewards+=seq_rewards
                    final_rewards.append(reward)
                    total_rewards.append(total_reward)
                    break

        avg_total_reward = sum(total_rewards) / len(total_rewards)
        avg_final_reward = sum(final_rewards) / len(final_rewards)
        avg_total_rewards.append(avg_total_reward)
        avg_final_rewards.append(avg_final_reward)
        prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

        rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)
        agent.learn(torch.stack(log_probs), torch.from_numpy(rewards))
    print(f"Average total reward of last {EPISODE_PER_BATCH*10} episode of pretraining " \
       + f"is {sum(avg_total_rewards[-10:])/10.}")
    print(f"Average final reward of last {EPISODE_PER_BATCH*10} episode of pretraining " \
       + f"is {sum(avg_final_rewards[-10:])/10.}")

In [6]:
import gym
def main_pre_train(env,device):
  network = PolicyGradientNetwork()
  agent = PolicyGradientAgent(network)
  policy_gradient_training(env,agent)
  torch.save(agent.network.state_dict(), "/content/pretrain_model.pth")


###Training

In [7]:
import torch.nn as nn
import torch.nn.functional as F
class Q_Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)

    def forward(self, state):
        hid = torch.tanh(self.fc1(state))
        hid = torch.tanh(self.fc2(hid))
        hid = torch.tanh(self.fc3(hid))
        hid = torch.tanh(self.fc4(hid))
        return self.fc5(hid)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
import random
from collections import deque
class Buffer():
  def __init__(self, capacity):
      self.memory = deque([], maxlen=capacity)
  def to_memory(self, new_data):
      self.memory.append(new_data)
  def __len__(self):
      return len(self.memory)
  def sample_batch(self, batch_size):
      return random.sample(self.memory, batch_size)

In [9]:
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
import gym
import math
import torch.optim as optim
from torch.distributions import Categorical
from collections import namedtuple

class Q_Learning_Process():
    def __init__(self, env, device, policy_net, target_net, buffer):
        self.env = env
        self.device = device
        self.policy_net = policy_net.to(self.device)
        self.target_net = target_net.to(self.device)
        self.buffer = buffer

    def _exploration(self,steps_done,state,EPS_START,EPS_END,EPS_DECAY):
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
        if sample > eps_threshold:
            with torch.no_grad():
              input = torch.FloatTensor(state).unsqueeze(0).to(self.device)
              return self.policy_net(input).max(1).indices.detach().item()

        else:
          return self.env.action_space.sample()

    def update_buffer(self,state,done,steps_done,EPS_START,EPS_END,EPS_DECAY):
        transition = namedtuple('transition_calss',('state', 'action', 'next_state', 'reward'))
        action = self._exploration(steps_done, state,EPS_START,EPS_END,EPS_DECAY)
        next_state, reward, done, info = self.env.step(action)
        if not done:
          data = transition(state,action,next_state,reward)
          self.buffer.to_memory(data)
        steps_done += 1
        return reward,next_state,done,steps_done

    def optimize_model(self, optimizer, LR, BATCH_SIZE, GAMMA):
        self.policy_net.train()
        batch = self.buffer.sample_batch(BATCH_SIZE)
        state = torch.FloatTensor([d.state for d in batch]).to(self.device)
        action = torch.LongTensor([d.action for d in batch]).view(-1,1).to(self.device)
        next_state = torch.FloatTensor([d.next_state for d in batch]).to(self.device)
        reward = torch.FloatTensor([d.reward for d in batch]).to(self.device)
        out = self.policy_net(state)

        Q = out.gather(1,action)

        with torch.no_grad():
          Q_next_max = self.target_net(next_state).max(1).values

        criterion = nn.SmoothL1Loss()
        loss = criterion(Q,(reward + GAMMA*Q_next_max).unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(self.policy_net.parameters(), 100)
        optimizer.step()
        return loss

    def eval(self,EPISODE_PER_BATCH):
        total_rewards = []
        final_rewards = []
        for e in range(EPISODE_PER_BATCH):
          rewards = 0
          done = False
          state = self.env.reset()
          while not done:
            input = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            action = \
            self.target_net(input).max(1).indices.detach().item()

            next_state, reward, done, info = self.env.step(action)
            rewards += reward

            if done:
              total_rewards.append(rewards)
              final_rewards.append(reward)
            state = next_state

        return sum(total_rewards)/len(total_rewards),\
            sum(final_rewards)/len(total_rewards)
    def soft_update(self,TAU):
      agent_target_net_state_dict = self.target_net.state_dict()
      agent_policy_net_state_dict = self.policy_net.state_dict()
      for key in agent_policy_net_state_dict:
          agent_target_net_state_dict[key] = \
          agent_policy_net_state_dict[key]*TAU + agent_target_net_state_dict[key]*(1-TAU)
      self.target_net.load_state_dict(agent_target_net_state_dict)

    def save(self, optimizer, PATH):
        Model_Dict = {
            "policy_net" : self.policy_net.state_dict(),
            "target_net" : self.target_net.state_dict(),
            "optimizer" : optimizer.state_dict(),
        }
        torch.save(Model_Dict, PATH)

    def load(self, optimizer, PATH):
        checkpoint = torch.load(PATH)
        self.policy_net.load_state_dict(checkpoint["policy_net"])
        self.target_net.load_state_dict(checkpoint["target_net"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        return optimizer

In [10]:
class LR_Scheduler():
    def __init__(self,LR,optimizer):
        self.LR = LR
        self.optimizer = optimizer
        self._step = 0
        self._rate = 0
    def step(self):
        self._step += 1
        if self._step<=200:
          self._rate = self.LR
        else:
          self._rate = self.LR * \
          (self._step-200) ** (-0.5)

        self.optimizer.param_groups[0]["lr"] = self._rate
        self.optimizer.step()
    def zero_grad(self):
        return self.optimizer.zero_grad()

    def state_dict(self):
        return self.optimizer.state_dict()

    def load_state_dict(self,state_dict):
        return self.optimizer.load_state_dict(state_dict)

    def set_step(self,step):
        self._step = step

In [11]:
def plot_data(is_ipython,loss,train_total_rewards,train_final_rewards,eval_total_rewards,eval_final_rewards):
  plt.clf()

  plt.figure(1)

  plt.title("Train")
  index = list(range(len(train_total_rewards)))
  plt.plot(index, train_total_rewards, c='tab:cyan', label='total rewards')
  plt.plot(index, train_final_rewards, c='tab:red', label='final rewards')
  plt.xlabel('Batch')
  plt.ylabel('Reward')
  plt.ylim(-300,200)
  if is_ipython:
    display.display(plt.gcf())

  plt.figure(2)

  plt.title("Eval")
  index = list(range(len(eval_total_rewards)))
  plt.plot(index, eval_total_rewards, c='tab:cyan', label='total rewards')
  plt.plot(index, eval_final_rewards, c='tab:red', label='final rewards')
  plt.xlabel('Batch')
  plt.ylabel('Reward')
  if is_ipython:
    display.display(plt.gcf())

  plt.figure(3)

  plt.title("Loss")
  index = list(range(len(loss)))
  plt.plot(index, loss, c='tab:cyan', label='total rewards')
  plt.xlabel('Batch')

  if is_ipython:
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [12]:
from tqdm.notebook import tqdm
import csv
import time

def train_eval(is_ipython,
        env,
        device,
        pretrain,
        LOAD,
        MODEL_PATH,
        TRAIN_REWARD_PATH,
        EVAL_REWARD_PATH,
        LOSS_PATH,
        SAVE_EPISODE,
        BUFFER_SIZE,
        LR,
        NUM_EPISODE,
        GAMMA,
        TAU,
        EPS_START,
        EPS_END,
        EPS_DECAY,
        BATCH_SIZE):
  start = time.time()

  buffer = Buffer(BUFFER_SIZE)
  agent = Q_Learning_Process(env,device,Q_Network(),Q_Network(),buffer)
  if pretrain :
    if LOAD:
      pretrain = False
      print(f"The model is load from {MODEL_PATH}, it doesn't require pretraining")
    main_pre_train(env,device)

  agent.policy_net.load_state_dict(torch.load("/content/pretrain_model.pth"))
  agent.target_net.load_state_dict(agent.policy_net.state_dict())

  train_total_rewards = []
  train_final_rewards = []
  eval_total_rewards = []
  eval_final_rewards = []
  avg_loss = []

  with open(TRAIN_REWARD_PATH, 'w') as f:
      writer = csv.writer(f)
      writer.writerow(['total_reward', 'final_reward'])
  with open(EVAL_REWARD_PATH, 'w') as f:
      writer = csv.writer(f)
      writer.writerow(['total_reward', 'final_reward'])

  steps_done = 0

  optimizer = LR_Scheduler(LR,optim.AdamW(agent.policy_net.parameters(), lr=LR, amsgrad=True))
  if LOAD :
    optimizer = agent.load(optimizer,MODEL_PATH)
    print(f"Load model from {MODEL_PATH}")

  prg_bar = tqdm(range(NUM_EPISODE))

  for episode_index in prg_bar:
    rewards = 0
    state = env.reset()
    done = False
    loss = []
    while not done :
      reward,next_state,done,steps_done = \
      agent.update_buffer(state,done,steps_done,EPS_START,EPS_END,EPS_DECAY)
      state = next_state
      rewards += reward

      if agent.buffer.__len__()>= BATCH_SIZE:
        now_loss = agent.optimize_model(optimizer,LR,BATCH_SIZE,GAMMA).detach().item()
        loss.append(now_loss)

        agent.soft_update(TAU)

    if len(loss)>0:
      avg_loss.append(sum(loss) / len(loss) )
    train_total_rewards.append(rewards)
    train_final_rewards.append(reward)


    if (episode_index+1) % SAVE_EPISODE ==0:
      # save model
      agent.save(optimizer,MODEL_PATH)
      print(f"Save model from {episode_index+1}th batch to {MODEL_PATH}")

      # save train reward
      total_rewards = list(zip(train_total_rewards,train_final_rewards))
      total_rewards = total_rewards[episode_index+1-SAVE_EPISODE:episode_index+1]
      with open(TRAIN_REWARD_PATH, 'a') as f:
        writer = csv.writer(f)
        writer.writerows(total_rewards)
      print(f"Save policy rewards from {(episode_index+2)-SAVE_EPISODE}th"+
         f"to {episode_index+1}th batchs to {TRAIN_REWARD_PATH}")

      # do eval and save eval reward
      eval_total_reward, eval_final_reward = agent.eval(5)
      eval_total_rewards.append(eval_total_reward)
      eval_final_rewards.append(eval_final_reward)

      with open(EVAL_REWARD_PATH, 'a') as f:
        writer = csv.writer(f)
        writer.writerow([eval_total_reward,eval_final_reward])

      print(f"Save target rewards of {episode_index+1}th batch to {TRAIN_REWARD_PATH}")

      # save loss
      with open(LOSS_PATH, 'a') as f:
        writer = csv.writer(f)
        writer.writerows([[x] for x in avg_loss[episode_index+1-SAVE_EPISODE:episode_index+1]])

      print(f"Save loss of {(episode_index+2)-SAVE_EPISODE}th "+
         f"to {episode_index+1}th batchs to {LOSS_PATH}")
      # plot data
      plot_data(is_ipython,avg_loss,train_total_rewards,train_final_rewards,eval_total_rewards,eval_final_rewards)

  end = time.time()
  print(f"total_spend_time = {end-start} seconds")

In [13]:
def inference(env,device,BUFFER_SIZE,LR,**kwargs):
  policy_net = Q_Network()
  target_net = Q_Network()
  buffer = Buffer(BUFFER_SIZE)
  agent = Q_Learning_Process(env,device,policy_net,target_net,buffer)
  agent.policy_net.load_state_dict(torch.load("/content/DQN_model.pth")["policy_net"])
  target_net.load_state_dict(policy_net.state_dict())
  optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

  agent.policy_net.eval()
  agent.target_net.eval()
  NUM_OF_TEST = 5
  action_list = []

  total_rewards = []
  final_rewards = []
  for i in range(NUM_OF_TEST):
    actions = []
    state = env.reset()

    img = plt.imshow(env.render(mode='rgb_array'))

    total_reward = 0
    env_as_nparray = env.render(mode='rgb_array')
    img = plt.imshow(env_as_nparray)
    done = False

    while not done:
      input = torch.FloatTensor(state).unsqueeze(0).to(device)
      input = torch.FloatTensor(state).unsqueeze(0).to(device)
      action = policy_net(input).max(1).indices.detach().item()
      img.set_data(env.render(mode='rgb_array'))
      display.display(plt.gcf())
      display.clear_output(wait=True)

      next_state, reward, done, info = env.step(action)
      total_reward += reward
      actions.append(action)
      if done:
        final_rewards.append(reward)

    total_rewards.append(total_reward)
    action_list.append(actions)
  print(f"Average total reward of {NUM_OF_TEST} test is {sum(total_rewards)/len(total_rewards)}")
  print(f"Average final reward if {NUM_OF_TEST} test is {sum(final_rewards)/len(final_rewards)}")

In [14]:
save_load ={
"LOAD" : False,
"SAVE_EPISODE" : 10,
"MODEL_PATH" : "/content/DQN_model.pth",
"TRAIN_REWARD_PATH" : "/content/DQN_train_rewards.csv",
"EVAL_REWARD_PATH" : "/content/DQN_eval_rewards.csv",
"LOSS_PATH" : "/content/DQN_loss.csv"
}
h_paras = {
"BUFFER_SIZE" : 10000,
"BATCH_SIZE" : 128,
"NUM_EPISODE" : 600,
"GAMMA" : 0.99,
"EPS_START" : 0.9,
"EPS_END" : 0.05,
"EPS_DECAY" : 500,
"TAU" : 0.005,
"LR" : 1e-3,
}

In [15]:
# %%capture
%matplotlib inline
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
def main(pretrain):
  env = gym.make('LunarLander-v2')
  fix(env, seed)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  train_eval(is_ipython,env,device,pretrain,**save_load,**h_paras)
  inference(env,device,**h_paras)

main(pretrain=True)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


  0%|          | 0/800 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


TypeError: bad operand type for unary +: 'str'